### Target Variable : Tip

In [1]:
!cd

C:\Users\i\Desktop\2021-K-Digital-Training-main\SQL


In [2]:
#!pip install PyMySQL==1.0.0

In [3]:
import pymysql.cursors
import pandas as pd
# Connect to the database

In [4]:
# localhost or 127.0.0.1

In [5]:
# MySQL DB에서 데이터 받아와서 DataFrame에 저장

conn = pymysql.connect(host='localhost', user='root', 
                       password='8475', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:

    with conn.cursor() as curs:
      sql = "SELECT * FROM tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)
      print(df)
      #df.to_csv('query.csv')

finally:

   conn.close()

     total_bill   tip     sex smoker  day    time  size
0         16.99  1.01  Female     No  Sun  Dinner   2.0
1         10.34  1.66    Male     No  Sun  Dinner   3.0
2         21.01  3.50    Male     No  Sun  Dinner   3.0
3         23.68  3.31    Male     No  Sun  Dinner   2.0
4         24.59  3.61  Female     No  Sun  Dinner   4.0
..          ...   ...     ...    ...  ...     ...   ...
247       20.65  3.35    Male     No       Dinner   4.0
248       20.65  3.35    Male     No       Dinner   5.0
249       10.34  1.66    None     No  Sun  Dinner   3.0
250       10.34  1.66    None     No  Sun  Dinner   NaN
251       16.99  1.01  Female     No  Sun    None   2.0

[252 rows x 7 columns]


In [6]:
df.isnull().sum()

total_bill    0
tip           0
sex           2
smoker        0
day           0
time          1
size          1
dtype: int64

### Encoding

In [7]:
df['sex'].replace({'Female':0, 'Male':1}, inplace=True)

In [8]:
df["smoker"].replace({"No" : 0, "Yes" : 1}, inplace=True)
df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3}, inplace=True)
df["time"].replace({"Lunch" : 0, "Dinner" : 1}, inplace=True)

In [9]:
import numpy as np

In [10]:
df.replace('',np.nan,inplace=True) # '' -> np.nan

In [11]:
df.isnull().sum()

total_bill    0
tip           0
sex           2
smoker        0
day           3
time          1
size          1
dtype: int64

In [12]:
df.describe()

,total_bill,tip,sex,smoker,day,time,size
count,252.000000,252.000000,250.000000,252.000000,249.000000,251.000000,251.000000
mean,19.702500,2.977460,0.644000,0.369048,1.746988,0.729084,2.589641
std,8.803699,1.378674,0.479776,0.483507,1.155388,0.445321,0.960694
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,13.347500,2.000000,0.000000,0.000000,1.000000,0.000000,2.000000
50%,17.795000,2.900000,1.000000,0.000000,2.000000,1.000000,2.000000
75%,24.022500,3.502500,1.000000,1.000000,3.000000,1.000000,3.000000
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000


In [13]:
type(df)

pandas.core.frame.DataFrame

In [14]:
#import numpy as np
from sklearn.impute import SimpleImputer

In [15]:
data = df.values

In [16]:
y=df['tip'] # Target variable -> Tip

In [17]:
X=df.drop('tip',axis=1)

In [18]:
X.columns

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [19]:
y.describe()

count    252.000000
mean       2.977460
std        1.378674
min        1.000000
25%        2.000000
50%        2.900000
75%        3.502500
max       10.000000
Name: tip, dtype: float64

In [20]:
data=X.values # Train data

In [21]:
y=y.values

In [22]:
sum(np.isnan(data).flatten())

7

In [23]:
sum(np.isnan(y).flatten())

0

In [24]:
imputer = SimpleImputer(strategy='median')

In [25]:
imputer.fit(data)

SimpleImputer(strategy='median')

In [26]:
# transform the dataset
data_trans = imputer.transform(data) # Imputed data set

### RFE

In [27]:
from sklearn.feature_selection import RFE

In [35]:
#from sklearn.tree import DecisionTreeClassifier

In [28]:
from sklearn.svm import SVR

In [29]:
estimator = SVR(kernel="linear")

In [30]:
# define RFE
rfe = RFE(estimator, n_features_to_select=4)

In [31]:
# fit RFE
selector=rfe.fit(data_trans, y)

In [69]:
!cd

C:\Users\i\Desktop\2021-K-Digital-Training-main\SQL


In [32]:
selector.support_

array([ True,  True, False, False,  True,  True])

In [33]:
# summarize all features
for i in range(data_trans.shape[1]):
  print('Column: %d,  Rank: %d' % (i,  rfe.ranking_[i]))

Column: 0,  Rank: 1
Column: 1,  Rank: 1
Column: 2,  Rank: 3
Column: 3,  Rank: 2
Column: 4,  Rank: 1
Column: 5,  Rank: 1


In [34]:
df1=pd.DataFrame(data_trans, columns=['total_bill', 'sex', 'smoker', 'day', 'time', 'size'])

In [35]:
df1.columns

In [37]:
df1.drop(['smoker', 'day'], axis=1)

,total_bill,sex,time,size
0,16.99,0.0,1.0,2.0
1,10.34,1.0,1.0,3.0
2,21.01,1.0,1.0,3.0
3,23.68,1.0,1.0,2.0
4,24.59,0.0,1.0,4.0
...,...,...,...,...
247,20.65,1.0,1.0,4.0
248,20.65,1.0,1.0,5.0
249,10.34,1.0,1.0,3.0
250,10.34,1.0,1.0,2.0


### PCA

In [39]:
from sklearn.decomposition import PCA

In [40]:
data_trans.shape

(252, 6)

In [41]:
# define the transform
trans = PCA(n_components=4)

In [42]:
# transform the data
X_dim = trans.fit_transform(data_trans)

In [43]:
# summarize data after the transform
print(X_dim[:3, :])

[[-2.72161111 -1.23805454 -0.43580267  0.80856409]
 [-9.28403422 -1.60709112  0.92640616 -0.25494631]
 [ 1.36070512 -1.30036284  0.29684274 -0.16815677]]


### Regression Feature Selection

In [44]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [45]:
fs = SelectKBest(score_func=f_regression, k=4)

In [46]:
data_trans.shape

(252, 6)

In [47]:
y.shape

(252,)

In [48]:
# apply feature selection
X_selected = fs.fit_transform(data_trans, y)

In [54]:
parameters = fs.get_params(deep=True)

In [66]:
parameters

{'k': 4,
 'score_func': <function sklearn.feature_selection._univariate_selection.f_regression(X, y, *, center=True)>}

In [59]:
parameters['score_func']

<function sklearn.feature_selection._univariate_selection.f_regression(X, y, *, center=True)>

In [49]:
print(X_selected.shape)

In [51]:
X_selected.shape[1]

4

In [64]:
df_rfe_selected = pd.DataFrame(X_selected, columns=['total_bill', 'day', 'time', 'size'])

In [65]:
df_rfe_selected.describe()

,total_bill,day,time,size
count,252.000000,252.000000,252.000000,252.000000
mean,19.702500,1.750000,0.730159,2.587302
std,8.803699,1.148791,0.444761,0.959497
min,3.070000,0.000000,0.000000,1.000000
25%,13.347500,1.000000,0.000000,2.000000
50%,17.795000,2.000000,1.000000,2.000000
75%,24.022500,3.000000,1.000000,3.000000
max,50.810000,3.000000,1.000000,6.000000
